In [21]:
import re
import os
import nltk
import string
import requests
import time

import numpy as np
import pandas as pd

from bs4 import BeautifulSoup
from nltk.corpus import wordnet
from collections import Counter
from nltk.corpus import stopwords

from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

from sklearn.decomposition import TruncatedSVD

import requests #(IP address connected + URL)
from sklearn.feature_extraction import _stop_words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation


nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /Users/nixi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/nixi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/nixi/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

# MyNLProject
To what extend does Natural language Processingdacilitate the analysis of the correlation betwenn product description and pricing in online jewellery retail? Acase study od Essential Jewellery.

The following code attempts to extract data from Pandora and Essential Jewellery. Althought the final goal for this project focusses on Essential Jewellery, I wanted to show all the exploratory techniques and processes in creating this finished work.
Adapted code from Week-5-Web-data-and-generative-text, available at: https://git.arts.ac.uk/tbroad/web-scrape-nursery-rhymes/commit/da77acf4524ab39b2476c1a6d5feddad888375f9.

# Working with the URL of the specifc ring on the Pandora website:

After each code correction, the URL for Pandora would not work, resulting in a 'failed to retrieve' message, as seen below.
Adapted code from Week-5-Web-data-and-generative-text, available at: https://git.arts.ac.uk/tbroad/web-scrape-nursery-rhymes/commit/da77acf4524ab39b2476c1a6d5feddad888375f9.

In [12]:
def extract_information_from_page(url2):
    response = requests.get(url2)
    if response.status_code ==200:
        soup = BeautifulSoup(response.content, 'html.parser')
        title = soup.find ('h1').text
        price = soup.find('span', {'class': 'value', 'data-auto': 'lblProductPrice'}).text
        print(f"Title: {title}, {price}")
    else:
        print(f"Failed to retrieve page {url2}")

specific_pandora_jewellry_url= 'https://uk.pandora.net/en/rings/rings/stackable-rings/classic-wishbone-ring/197790CZ.html#navigation' 
extract_information_from_page(specific_pandora_jewellry_url)

Failed to retrieve page https://uk.pandora.net/en/rings/rings/stackable-rings/classic-wishbone-ring/197790CZ.html#navigation


In [13]:
# In this code block, i tried scraping Pandora's website.
specific_pandora_jewellry_url= 'https://uk.pandora.net/en/rings/rings/stackable-rings/classic-wishbone-ring/197790CZ.html#navigation' 

def extract_ring_price(specific_pandora_jewellry_url):
    response = requests.get(specific_pandora_jewellry_url)
    if response.status_code != 200:
        return "Failed to retrieve the page."
    
    soup = BeautifulSoup(response.text, 'html.parser')
    price_element = soup.find('span', {'class': 'value', 'data-auto': 'lblProductPrice'}) #class found using Inspect element tool
    if price_element:
        price_text = price_element.text
        price = float (price_text.replace('£', ''))
        return price
    else:
        return None
ring_price = extract_ring_price(specific_pandora_jewellry_url)
print(ring_price)
time.sleep(10)

Failed to retrieve the page.


NameError: name 'time' is not defined

# Why "Failed to retrieve the page"?
The above code produces a 'Failed to retreive the page'for a number of reasons:
(1) URL may be inaccesible from my browser or the URL may be incorrect for the ring i am after
(2) The website maybe blocking any further web scraping activities. if this is the case, I would need to incorporate user-agent headers to prevent scape blocing from the website. (https://scrapfly.io/blog/user-agent-header-in-web-scraping/)

User Agent code (below) created alongside John Watson Rooney at: https://www.youtube.com/watch?v=90t9WkQbQ2E. The code produces and output of '200', verifying that I am not a robot. The code should now work on Pandora's website. Although i had found a way to access the pandora site, I decided to reduce my proposed framework, using two e-commerce sites, for a focussed appraoch.

In [ ]:
import requests

headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
}

r = requests.get('https://uk.pandora.net/en/rings/rings/stackable-rings/classic-wishbone-ring/197790CZ.html#navigation', headers=headers)
r.status_code

200

Upon culminating this code, I was unsure whether the two websites are built the same. if not, then the code above which compares the class 'product-specification' will only have one value. If the class containing the price of the specific ring is different on the Pandora URL, the above code would need to be updates to reflect the new class name for Pandora.
   (1) Test the scraping code for the price data on Essential jewellry website , use print() to see if the correct data reflects.
    (2) Test same code for Pandora website.

# Scraping ring data from Essential Jewellry
I wanted to creating code which iterates through each Essential Jewellry website, logs all the rings then creates a list with the name and price. Code adapted from Week-2-Text-as-numbers, available at: https://git.arts.ac.uk/tbroad/NLP-23-24/blob/main/Week-2-Text-as-numbers/NLP%20Week%202.1-Text%20as%20Numbers-class.ipynb.


In [14]:
import time
import requests #(IP address connected + URL)
from bs4 import BeautifulSoup
import re
import requests
import pandas as pd
import numpy as np


title = []
price = []

#jewellry type: rings
rings = []
for page in range(1, 15): #there are 13 pages in total
    url1 = f'https://essentialjewellery.com/collections/all-rings' #stores website to variable url.

    # retrieve the HTML content...
    response = requests.get(url1)
    
    html_content = response.content

    # parse the HTML content using BeautifulSoup...
    soup = BeautifulSoup(html_content, 'html.parser')

# Find  product items and extract the price and title
    product_items = soup.find_all( class_='product-item') #class found using by analysing the structure of the web HTML.
    for item in product_items:
        product_name = item.find('a', class_='product-item-meta__title').text
        product_price = item.find('span', class_='price').text
        
             #append the above ring list to add the price and product name
        rings.append([product_name, product_price]) # the 'ring' variable contains a list of all the rings and their sale price.

print(rings) #creates a name and price colomn
time.sleep(1) #code from https://www.geeksforgeeks.org/sleep-in-python/. This code adds a delay in my request for pricing data, Essential's website has a terms of service which mentions to not overload servers with too many requests. Without the following code,my code produced an error output.

#find out how many listing there are and compare this data with the data on the URL. Helps validate my code.
number_of_products = len(rings)
print(f'Total number of products: {number_of_products}')


#             #Turn ring list into data frame outside the for loop#
#         #Code adapted from: https://www.blog.datahut.co/post/web-scraping-in-python and 
# df = pd.DataFrame(rings, columns=['Product Name', 'Sale price'])

#             #Turn the data frame into a csv file#
# df.to_csv('essential_jewellry_data.csv', index= False)
# print(df)
        


[['Sterling Silver Tiny 1.5mm Plain Band Ring', '\nSale price£9.00'], ['Sterling Silver Mini Cutout Heart Ring', '\nSale price£6.00'], ['Sterling Silver and Oval Gemstone Ring', '\nSale priceFrom £34.00\n'], ['Sterling Silver Organic Split Band Ring', '\nSale price£14.00'], ['Sterling Silver Three Band Hammered Spinner Ring', '\nSale price£32.00'], ['Sterling Silver 3 x 2mm Russian Wedding Ring', '\nSale price£20.00'], ['Sterling Silver 1.5mm Flat Band Ring', '\nSale price£8.00'], ['Sterling Silver and Cubic Zirconia Wishbone Ring', '\nSale price£12.99'], ['Sterling Silver and Cubic Zirconia Marquise Band Ring', '\nSale price£18.00'], ['Sterling Silver Narrow Hammered Band', '\nSale price£10.00'], ['Sterling Silver Curved Wishbone Ring', '\nSale price£8.00'], ['Sterling Silver 3 x 1mm Russian Wedding Band', '\nSale price£12.00'], ['Sterling Silver Heart Gemstone Bobble Ring', '\nSale priceFrom £12.00\n'], ['3mm Plain Sterling Silver D Shaped Band Ring', '\nSale price£16.00'], ['Sterlin

In [17]:

# jewellry type: wristwear
wristwear = []
for page in range(1, 8): #there are 7 pages in total
    url2 = f'https://essentialjewellery.com/collections/all-wristwear' #stores website to variable url.

    # retrieve the HTML content from url.
    response = requests.get(url2)
    
    html_content = response.content

    # parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')

# Find  product items and extract the price and title
    product_items = soup.find_all( class_='product-item') #class found using by analysing the structure of the web HTML.
    for item in product_items:
        product_name = item.find('a', class_='product-item-meta__title').text
        product_price = item.find('span', class_='price').text
        
             #append the above ring list to add the price and product name
        wristwear.append([product_name, product_price]) # the 'ring' variable contains a list of all the rings and their sale price.

print(wristwear) #creates a name and price colomn
time.sleep(1) #code from https://www.geeksforgeeks.org/sleep-in-python/. This code adds a delay in my request for pricing data, Essential's website has a terms of service which mentions to not overload servers with too many requests. Without the following code,my code produced an error output.

#find out how many listing there are and compare this data with the data on the URL. Helps validate my code.
number_of_products = len(wristwear)
print(f'Total number of products: {number_of_products}')


[['Sterling Silver and Gemstone Marquise Bracelet', '\nSale priceFrom £38.00\n'], ['Sterling Silver Daisy Chain Bracelet', '\nSale price£18.00'], ['Sterling Silver Ribbon Twist Bangle', '\nSale price£35.00'], ['Sterling Silver Braided Chain Bracelet', '\nSale price£15.00'], ['Sterling Silver Flat Star Link Bracelet', '\nSale price£9.00'], ['Sterling Silver and Gemstone Beaded Bracelet', '\nSale price£15.00'], ['Sterling Silver Fine Herringbone Bracelet', '\nSale price£12.00'], ['Sterling Silver Seed Link Slider Bracelet', '\nSale price£15.00'], ['Sterling Silver Intertwined Stations Bracelet', '\nSale price£22.00'], ['Sterling Silver Fine Bobble Bead Bracelet', '\nSale price£9.00'], ['Sterling Silver Simple Bobble Snake Chain Bracelet', '\nSale price£10.00'], ['Sterling Silver Paperclip Link Bracelet', '\nSale price£22.00'], ['Sterling Silver and Cubic Zirconia Contrast Lovers Link Bracelet', '\nSale price£28.00'], ['Sterling Silver Light Hammered Court Bangle', '\nSale price£24.00'], 

In [18]:

# jewellry type: earrings
earrings = []
for page in range(1, 29): #there are 28 pages in total
    url3 = f'https://essentialjewellery.com/collections/all-earrings' #stores website to variable url.

    # retrieve the HTML content from url.
    response = requests.get(url3)
    
    html_content = response.content

    # parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')

# Find  product items and extract the price and title
    product_items = soup.find_all( class_='product-item') #class found using by analysing the structure of the web HTML.
    for item in product_items:
        product_name = item.find('a', class_='product-item-meta__title').text
        product_price = item.find('span', class_='price').text
        
             #append the above ring list to add the price and product name
        earrings.append([product_name, product_price]) # the 'ring' variable contains a list of all the rings and their sale price.

print(earrings) #creates a name and price colomn
time.sleep(1) #code from https://www.geeksforgeeks.org/sleep-in-python/. This code adds a delay in my request for pricing data, Essential's website has a terms of service which mentions to not overload servers with too many requests. Without the following code,my code produced an error output.

#find out how many listing there are and compare this data with the data on the URL. Helps validate my code.
number_of_products = len(earrings)
print(f'Total number of products: {number_of_products}')


[['Sterling Silver 10mm Sleeper Hoop', '\nSale price£2.50'], ['Sterling Silver 3mm Bead Studs', '\nSale price£3.99'], ['Sterling Silver Essential Hinged Huggie Hoops', '\nSale priceFrom £12.00\n'], ['Sterling Silver 12mm Sleeper Hoop', '\nSale price£3.00'], ['Sterling Silver 8mm Sleeper Hoop', '\nSale priceFrom £2.25\n'], ['Sterling Silver and Gemstone Teardrop Earrings', '\nSale price£14.00'], ['Sterling Silver Claw Set Cubic Zirconia Studs', '\nSale price£5.99'], ['Sterling Silver 2mm Bead Studs', '\nSale price£3.99'], ['Sterling Silver Tiny Crossover Hoops', '\nSale price£12.00'], ['Sterling Silver Simple Knot Studs', '\nSale price£6.00'], ['Sterling Silver and Cubic Zirconia 12mm Huggy Hoops', '\nSale price£18.00'], ['Sterling Silver Huggie Hoops with Mismatched Celestial Charms', '\nSale price£16.00'], ['Sterling Silver 3mm Claw Set Cubic Zirconia Studs', '\nSale price£8.00'], ['Sterling Silver 10mm Square Tube Hoops', '\nSale price£8.00'], ['Sterling Silver Round Opal Studs', '\n

In [19]:
#jewellry type: neckwear

neckwear = []
for page in range(1, 15):
    url4 =  f'https://essentialjewellery.com/collections/all-neckwear'
 # retrieve the HTML content from url.
    response = requests.get(url3)
    
    html_content = response.content

    # parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')

# Find  product items and extract the price and title
    product_items = soup.find_all( class_='product-item') #class found using by analysing the structure of the web HTML.
    for item in product_items:
        product_name = item.find('a', class_='product-item-meta__title').text
        product_price = item.find('span', class_='price').text
        
             #append the above ring list to add the price and product name
        neckwear.append([product_name, product_price]) # the 'ring' variable contains a list of all the rings and their sale price.

print(neckwear) #creates a name and price colomn
time.sleep(1) #code from https://www.geeksforgeeks.org/sleep-in-python/. This code adds a delay in my request for pricing data, Essential's website has a terms of service which mentions to not overload servers with too many requests. Without the following code,my code produced an error output.

#find out how many listing there are and compare this data with the data on the URL. Helps validate my code.
number_of_products = len(neckwear)
print(f'Total number of products: {number_of_products}')

[['Sterling Silver 10mm Sleeper Hoop', '\nSale price£2.50'], ['Sterling Silver 3mm Bead Studs', '\nSale price£3.99'], ['Sterling Silver Essential Hinged Huggie Hoops', '\nSale priceFrom £12.00\n'], ['Sterling Silver 12mm Sleeper Hoop', '\nSale price£3.00'], ['Sterling Silver 8mm Sleeper Hoop', '\nSale priceFrom £2.25\n'], ['Sterling Silver and Gemstone Teardrop Earrings', '\nSale price£14.00'], ['Sterling Silver Claw Set Cubic Zirconia Studs', '\nSale price£5.99'], ['Sterling Silver 2mm Bead Studs', '\nSale price£3.99'], ['Sterling Silver Tiny Crossover Hoops', '\nSale price£12.00'], ['Sterling Silver Simple Knot Studs', '\nSale price£6.00'], ['Sterling Silver and Cubic Zirconia 12mm Huggy Hoops', '\nSale price£18.00'], ['Sterling Silver Huggie Hoops with Mismatched Celestial Charms', '\nSale price£16.00'], ['Sterling Silver 3mm Claw Set Cubic Zirconia Studs', '\nSale price£8.00'], ['Sterling Silver 10mm Square Tube Hoops', '\nSale price£8.00'], ['Sterling Silver Round Opal Studs', '\n

In [20]:

body_jewellry = []
for page in range(1, 6):
    url5 =  f'https://essentialjewellery.com/collections/all-body-jewellery'
 
 # retrieve the HTML content from url.
    response = requests.get(url5)
    
    html_content = response.content

    # parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')

# Find  product items and extract the price and title
    product_items = soup.find_all( class_='product-item') #class found using by analysing the structure of the web HTML.
    for item in product_items:
        product_name = item.find('a', class_='product-item-meta__title').text
        product_price = item.find('span', class_='price').text
        
             #append the above ring list to add the price and product name
        body_jewellry.append([product_name, product_price]) # the 'ring' variable contains a list of all the rings and their sale price.

print(body_jewellry) #creates a name and price colomn
time.sleep(1) #code from https://www.geeksforgeeks.org/sleep-in-python/. This code adds a delay in my request for pricing data, Essential's website has a terms of service which mentions to not overload servers with too many requests. Without the following code,my code produced an error output.

#find out how many listing there are and compare this data with the data on the URL. Helps validate my code.
number_of_products = len(body_jewellry)
print(f'Total number of products: {number_of_products}')

[['Sterling Silver 7mm Plain Nose Hoop', '\nSale price£2.50'], ['Surgical Steel Labret with 3mm Claw Set Crystal', '\nSale price£5.00'], ['Sterling Silver 2mm Crystal Nose Stud', '\nSale price£2.50'], ['Sterling Silver 8mm Plain Nose Hoop', '\nSale price£2.50'], ['Sterling Silver 7mm Nose Hoop with Ball', '\nSale price£2.50'], ['Sterling Silver and Clear Crystal Square Set  Nose Stud', '\nSale price£2.50'], ['Sterling Silver 1mm Bead Nose Stud', '\nSale price£2.50'], ['Surgical Steel Crystal Edge Hoop', '\nSale priceFrom £10.00\n'], ['Sterling Silver Light Ball and Twisted Chain Anklet', '\nSale price£9.00'], ['Sterling Silver Double Ring Illusion Nose Stud', '\nSale price£3.50'], ['Surgical Steel Hinged Segment Ring', '\nSale price£6.00'], ['Sterling Silver Tiny Hammered Star Nose Stud', '\nSale price£2.50'], ['Surgical Steel Heart Twist Ring', '\nSale price£4.00'], ['Surgical Steel Labret with 2mm Claw Set Crystal', '\nSale price£5.00'], ['Surgical Steel and Crystal Marquise Petals L

In [21]:

chains = []
for page in range(1, 2):
    url6 =  f'https://essentialjewellery.com/collections/cartilage-and-helix'
 # retrieve the HTML content from url.
    response = requests.get(url6)
    
    html_content = response.content

    # parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')

# Find  product items and extract the price and title
    product_items = soup.find_all( class_='product-item') #class found using by analysing the structure of the web HTML.
    for item in product_items:
        product_name = item.find('a', class_='product-item-meta__title').text
        product_price = item.find('span', class_='price').text
        
             #append the above ring list to add the price and product name
        chains.append([product_name, product_price]) # the 'ring' variable contains a list of all the rings and their sale price.

print(chains) #creates a name and price colomn
time.sleep(1) #code from https://www.geeksforgeeks.org/sleep-in-python/. This code adds a delay in my request for pricing data, Essential's website has a terms of service which mentions to not overload servers with too many requests. Without the following code,my code produced an error output.

#find out how many listing there are and compare this data with the data on the URL. Helps validate my code.
number_of_products = len(chains)
print(f'Total number of products: {number_of_products}')

[['Surgical Steel Labret with 2mm Claw Set Crystal', '\nSale price£5.00'], ['Surgical Steel Labret with 3mm Claw Set Crystal', '\nSale price£5.00'], ['2.5mm Claw Set Crystal Ear Bar', '\nSale price£3.99'], ['3mm Claw Set Crystal Ear Bar', '\nSale price£3.99'], ['Surgical Steel and Crystal Triple Stone Hoop', '\nSale price£8.00'], ['Surgical Steel Set Crystal Hinged Segment Ring', '\nSale price£8.00'], ['Surgical Steel Crystal Edge Hoop', '\nSale priceFrom £10.00\n'], ['Surgical Steel Hinged Segment Ring', '\nSale price£6.00'], ['Surgical Steel Lightning Bolt Labret', '\nSale price£4.99'], ['Surgical Steel Moon and Star Labret', '\nSale price£5.99'], ['Surgical Steel Star Labret', '\nSale price£4.99'], ['Surgical Steel Triple Star Ear Bar', '\nSale price£4.99'], ['Surgical Steel Bobble Twist Ring', '\nSale price£4.00'], ['Surgical Steel Heart Twist Ring', '\nSale price£4.00'], ['Surgical Steel Snakey Cartilage Hoop', '\nSale price£7.00'], ['Surgical Steel Ball Closure Ring', '\nSale pri

# Making the above code concise
All data is structured the same and can be cleaned using the same regex rules after analysing the above data sets.
How can I condense the amount of code written using a for loop?

In [3]:
import time
import requests #(IP address connected + URL)
from bs4 import BeautifulSoup
import re
import requests
import pandas as pd
import numpy as np
# list all jewellry from which all HTML data will be stored, depending on the category of jewellry.

rings = []
earrings =[]
wristwear =[]
neckwear =[]
body_jewellry =[]
chains =[]

#call rings: 
for page in range(1, 15): #there are 13 pages in total
    url1 = f'https://essentialjewellery.com/collections/all-rings' #stores website to variable url.

    # retrieve the HTML content...
    response = requests.get(url1)
    
    html_content = response.content

    # parse the HTML content using BeautifulSoup...
    soup = BeautifulSoup(html_content, 'html.parser')

# Find  product items and extract the price and title
    product_items = soup.find_all( class_='product-item') #class found using by analysing the structure of the web HTML.
    for item in product_items:
        product_name = item.find('a', class_='product-item-meta__title').text
        product_price = item.find('span', class_='price').text
        
             #append the above ring list to add the price and product name
        rings.append([product_name, product_price]) # the 'ring' variable contains a list of all the rings and their sale price.

    time.sleep(1) #code from https://www.geeksforgeeks.org/sleep-in-python/. This code adds a delay in my request for pricing data, Essential's website has a terms of service which mentions to not overload servers with too many requests. Without the following code,my code produced an error output.


    #call wristwear:
for page in range(1, 8): #there are 7 pages in total
    url2 = f'https://essentialjewellery.com/collections/all-wristwear' #stores website to variable url.

        # retrieve the HTML content from url.
    response = requests.get(url2)
        
    html_content = response.content

        # parse the HTML 
    soup = BeautifulSoup(html_content, 'html.parser')

    # Find product items and extract the price and title
    product_items = soup.find_all( class_='product-item') #class found using by analysing the structure of the web HTML.
    for item in product_items:
        product_name = item.find('a', class_='product-item-meta__title').text
        product_price = item.find('span', class_='price').text
            
                #append the above ring list to add the price and product name
        wristwear.append([product_name, product_price]) # the 'ring' variable contains a list of all the rings and their sale price.

    time.sleep(1) 


    #call earrings:
for page in range(1, 29): #there are 28 pages in total
    url3 = f'https://essentialjewellery.com/collections/all-earrings' #stores website to variable url.

        # retrieve the HTML content from url.
    response = requests.get(url3)
        
    html_content = response.content

        # parse the HTML 
    soup = BeautifulSoup(html_content, 'html.parser')

    # Find product items and extract the price and title
    product_items = soup.find_all( class_='product-item') #class found using by analysing the structure of the web HTML.
    for item in product_items:
        product_name = item.find('a', class_='product-item-meta__title').text
        product_price = item.find('span', class_='price').text
            
                #append the above ring list to add the price and product name
        earrings.append([product_name, product_price]) # the 'ring' variable contains a list of all the rings and their sale price.

        time.sleep(1)


    #call neckwear
for page in range(1, 15):
    url4 =  f'https://essentialjewellery.com/collections/all-neckwear'
 # retrieve the HTML content from url.
    response = requests.get(url3)
    
    html_content = response.content

    # parse HTML 
    soup = BeautifulSoup(html_content, 'html.parser')

# Find  product items and extract the price and title
    product_items = soup.find_all( class_='product-item') #class found using by analysing the structure of the web HTML.
    for item in product_items:
        product_name = item.find('a', class_='product-item-meta__title').text
        product_price = item.find('span', class_='price').text
        
             #append the above ring list to add the price and product name
        neckwear.append([product_name, product_price])
        time.sleep(1)


#call body_jewellry
for page in range(1, 6):
    url5 =  f'https://essentialjewellery.com/collections/all-body-jewellery'
 
 # retrieve HTML content
    response = requests.get(url5)
    
    html_content = response.content

    # parse 
    soup = BeautifulSoup(html_content, 'html.parser')

# Find  product items and extract the price and title
    product_items = soup.find_all( class_='product-item') #class found using by analysing the structure of the web HTML.
    for item in product_items:
        product_name = item.find('a', class_='product-item-meta__title').text
        product_price = item.find('span', class_='price').text
        
             #append the above ring list to add the price and product name
        body_jewellry.append([product_name, product_price])
        time.sleep(1)


#call chains
for page in range(1, 2):
    url6 =  f'https://essentialjewellery.com/collections/cartilage-and-helix'
 # retrieve the HTML content from url.
    response = requests.get(url6)
    
    html_content = response.content

    # parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')

# Find  product items and extract the price and title
    product_items = soup.find_all( class_='product-item') #class found using by analysing the structure of the web HTML.
    for item in product_items:
        product_name = item.find('a', class_='product-item-meta__title').text
        product_price = item.find('span', class_='price').text
        
             #append the above ring list to add the price and product name
        chains.append([product_name, product_price]) # the 'ring' variable contains a list of all the rings and their sale price.
        time.sleep(1)


df_rings = pd.DataFrame(rings, columns=['Product Name', 'Sale price'])
df_earrings= pd.DataFrame(earrings, columns=['Product Name', 'Sale price'])
df_wristwear= pd.DataFrame(wristwear, columns=['Product Name', 'Sale price'])
df_neckwear= pd.DataFrame(neckwear, columns=['Product Name', 'Sale price'])
df_body_jewellry= pd.DataFrame(body_jewellry, columns=['Product Name', 'Sale price'])
df_chains= pd.DataFrame(chains, columns=['Product Name', 'Sale price'])

# Combine the above data using pandas. code adapted from https://realpython.com/pandas-merge-join-and-concat/.
dataframe =[df_rings,df_earrings,df_wristwear,df_neckwear, df_body_jewellry, df_chains ]
df_combine = pd.concat (dataframe, ignore_index=True) #used an 'ignore_index',within the concatination function, which creates a Boolean value. 

print(df_combine)

                                          Product Name  \
0           Sterling Silver Tiny 1.5mm Plain Band Ring   
1               Sterling Silver and Oval Gemstone Ring   
2               Sterling Silver Mini Cutout Heart Ring   
3              Sterling Silver Organic Split Band Ring   
4     Sterling Silver Three Band Hammered Spinner Ring   
...                                                ...   
1651       Surgical Steel and Crystal Pave Disc Labret   
1652   Surgical Steel Synthetic Opal Mini Stone Labret   
1653   Surgical Steel Opal Effect Crescent Moon Labret   
1654         Surgical Steel Labret with Single Crystal   
1655                             Surgical Steel Labret   

                     Sale price  
0             \nSale price£9.00  
1     \nSale priceFrom £34.00\n  
2             \nSale price£6.00  
3            \nSale price£14.00  
4            \nSale price£32.00  
...                         ...  
1651          \nSale price£6.99  
1652          \nSale price£4.99

In [4]:
rings

[['Sterling Silver Tiny 1.5mm Plain Band Ring', '\nSale price£9.00'],
 ['Sterling Silver and Oval Gemstone Ring', '\nSale priceFrom £34.00\n'],
 ['Sterling Silver Mini Cutout Heart Ring', '\nSale price£6.00'],
 ['Sterling Silver Organic Split Band Ring', '\nSale price£14.00'],
 ['Sterling Silver Three Band Hammered Spinner Ring', '\nSale price£32.00'],
 ['Sterling Silver 3 x 2mm Russian Wedding Ring', '\nSale price£20.00'],
 ['Sterling Silver 1.5mm Flat Band Ring', '\nSale price£8.00'],
 ['Sterling Silver and Cubic Zirconia Wishbone Ring', '\nSale price£12.99'],
 ['Sterling Silver and Cubic Zirconia Marquise Band Ring',
  '\nSale price£18.00'],
 ['Sterling Silver Narrow Hammered Band', '\nSale price£10.00'],
 ['Sterling Silver Curved Wishbone Ring', '\nSale price£8.00'],
 ['Sterling Silver Heart Gemstone Bobble Ring', '\nSale priceFrom £12.00\n'],
 ['Sterling Silver 3 x 1mm Russian Wedding Band', '\nSale price£12.00'],
 ['3mm Plain Sterling Silver D Shaped Band Ring', '\nSale price£16.

In [5]:
earrings

[['Sterling Silver 10mm Sleeper Hoop', '\nSale price£2.50'],
 ['Sterling Silver 3mm Bead Studs', '\nSale price£3.99'],
 ['Sterling Silver Essential Hinged Huggie Hoops',
  '\nSale priceFrom £12.00\n'],
 ['Sterling Silver 12mm Sleeper Hoop', '\nSale price£3.00'],
 ['Sterling Silver 8mm Sleeper Hoop', '\nSale priceFrom £2.25\n'],
 ['Sterling Silver and Gemstone Teardrop Earrings', '\nSale price£14.00'],
 ['Sterling Silver Claw Set Cubic Zirconia Studs', '\nSale price£5.99'],
 ['Sterling Silver 2mm Bead Studs', '\nSale price£3.99'],
 ['Sterling Silver Tiny Crossover Hoops', '\nSale price£12.00'],
 ['Sterling Silver Simple Knot Studs', '\nSale price£6.00'],
 ['Sterling Silver and Cubic Zirconia 12mm Huggy Hoops', '\nSale price£18.00'],
 ['Sterling Silver 3mm Claw Set Cubic Zirconia Studs', '\nSale price£8.00'],
 ['Sterling Silver Huggie Hoops with Mismatched Celestial Charms',
  '\nSale price£16.00'],
 ['Sterling Silver 10mm Square Tube Hoops', '\nSale price£8.00'],
 ['Sterling Silver Roun

# Putting the current data into a csv file.
This helped me to see all my data in one place and what words ans symbols i would need to remove.
Code adapted from  https://stackoverflow.com/questions/59422448/with-file-openr-encoding-utf-8-as-f-attributeerror-str-object-has-no-a

In [6]:
df_combine.to_csv('essential_jewelry_data.csv', index=False, encoding='utf-8')

# Cleaning data
I origionally removed 'ring' and 'sterling silver from my data set when i was just worjking with the ring data set. However, this is no longer a viable method of stop-word removal. Essential Jewellry has gold and cord jewellry, removing 'sterling silver' removes a key element of the product description which may have some correlation with the price of the jewellry in my later classification stages. Instead I opted to clean the 'Sale price' column, removing unwanted characters and extract only the numerical part, as seen in week 6 classification lecture (https://git.arts.ac.uk/tbroad/NLP-23-24/blob/main/Week-6-Classification/classification-lecture.ipynb) 
and  https://git.arts.ac.uk/tbroad/NLP-23-24/blob/main/Week-3-Manipulating-text/bonus-exercise-regex.ipynb.

# Tokenisation
Code adapted from Yasuni Chamodya at: https://stackoverflow.com/questions/33098040/how-to-use-word-tokenize-in-data-frame. 



In [35]:
#first convert al values within data set into lower case...
df = pd.read_csv('essential_jewelry_data.csv')

df[['Product Name', 'Sale price']] = df[['Product Name', 'Sale price']].apply(lambda x: x.str.lower()) # Convert all data to lowercase

print(df) 


                                          Product Name  \
0           sterling silver tiny 1.5mm plain band ring   
1               sterling silver and oval gemstone ring   
2               sterling silver mini cutout heart ring   
3              sterling silver organic split band ring   
4     sterling silver three band hammered spinner ring   
...                                                ...   
1651       surgical steel and crystal pave disc labret   
1652   surgical steel synthetic opal mini stone labret   
1653   surgical steel opal effect crescent moon labret   
1654         surgical steel labret with single crystal   
1655                             surgical steel labret   

                     Sale price  
0             \nsale price£9.00  
1     \nsale pricefrom £34.00\n  
2             \nsale price£6.00  
3            \nsale price£14.00  
4            \nsale price£32.00  
...                         ...  
1651          \nsale price£6.99  
1652          \nsale price£4.99

In [36]:
#tokenise using nltk tokenize
df['tokenized_sale_price'] = df['Sale price'].apply(word_tokenize)
df['tokenized_product_name'] = df['Product Name'].apply(word_tokenize)

print(df)

                                          Product Name  \
0           sterling silver tiny 1.5mm plain band ring   
1               sterling silver and oval gemstone ring   
2               sterling silver mini cutout heart ring   
3              sterling silver organic split band ring   
4     sterling silver three band hammered spinner ring   
...                                                ...   
1651       surgical steel and crystal pave disc labret   
1652   surgical steel synthetic opal mini stone labret   
1653   surgical steel opal effect crescent moon labret   
1654         surgical steel labret with single crystal   
1655                             surgical steel labret   

                     Sale price       tokenized_sale_price  \
0             \nsale price£9.00         [sale, price£9.00]   
1     \nsale pricefrom £34.00\n  [sale, pricefrom, £34.00]   
2             \nsale price£6.00         [sale, price£6.00]   
3            \nsale price£14.00        [sale, price£14.

# Remove stop words within the price column
 The above code worked well to tokenise the column 'Product Name', however the 'Sale Price' column still had some unwanted text within, for example 'Sale', 'price', 'From'.
- Code adapted from: https://git.arts.ac.uk/tbroad/NLP-23-24/blob/main/Week-2-Text-as-numbers/bonus-exercise-solutions.ipynb
- Additionally removing the pound sign from tokenized_sale_price. Only the numerical value is needed for later classification between expensice and not expensice jewellry.
- Use the scrollable element in the code output to see changes

In [37]:
def custom_tokenize(text):
    
    tokens = re.findall(r'\b[\w.]+\b', text) #code adapted from https://www.codecademy.com/resources/docs/python/regex/findall.
    
    # Looking at the dataset, I wanted to remove the following words...
    words_to_remove = {'and', 'sale', 'pricefrom', 'price', 'with'}


    # # I added a stemmer to remove word duplicates (see data under 'Feature extraction...' for more detail)
    # Remove 'hoops' with 'hoop' to avoid duplicates within the data. 

    replaced_tokens = ['hoop' if word == 'hoops' else word for word in tokens if word not in words_to_remove]
    
    return replaced_tokens
    tokens_without_stop_words = [word for word in tokens if word not in words_to_remove]


# apply custom tokenization function to the 'Sale price' column...
df['Sale price'] = df['Sale price'].apply(custom_tokenize)

# apply for 'Product Name'...
df['Product Name'] = df['Product Name'].apply(custom_tokenize)

# Check for unique words after removing stop words
tokens_without_stop_words = [word for tokens in df['tokenized_product_name'] for word in tokens]
stop_vocab = np.unique(tokens_without_stop_words)
print("unique words", len(stop_vocab))
print (df)

unique words 150
                                           Product Name Sale price  \
0     [sterling, silver, tiny, 1.5mm, plain, band, r...     [9.00]   
1              [sterling, silver, oval, gemstone, ring]    [34.00]   
2         [sterling, silver, mini, cutout, heart, ring]     [6.00]   
3        [sterling, silver, organic, split, band, ring]    [14.00]   
4     [sterling, silver, three, band, hammered, spin...    [32.00]   
...                                                 ...        ...   
1651     [surgical, steel, crystal, pave, disc, labret]     [6.99]   
1652  [surgical, steel, synthetic, opal, mini, stone...     [4.99]   
1653  [surgical, steel, opal, effect, crescent, moon...     [4.99]   
1654         [surgical, steel, labret, single, crystal]     [5.99]   
1655                          [surgical, steel, labret]     [3.99]   

           tokenized_sale_price  \
0            [sale, price£9.00]   
1     [sale, pricefrom, £34.00]   
2            [sale, price£6.00]   
3 

In [131]:
df #double check what my dataframe looks like with all the above data cleaning

,Product Name,Sale price,tokenized_sale_price,tokenized_product_name
0,"[sterling, silver, tiny, 1.5mm, plain, band, r...",[9.00],"[sale, price£9.00]","[sterling, silver, tiny, 1.5mm, plain, band, r..."
1,"[sterling, silver, oval, gemstone, ring]",[34.00],"[sale, pricefrom, £34.00]","[sterling, silver, and, oval, gemstone, ring]"
2,"[sterling, silver, mini, cutout, heart, ring]",[6.00],"[sale, price£6.00]","[sterling, silver, mini, cutout, heart, ring]"
3,"[sterling, silver, organic, split, band, ring]",[14.00],"[sale, price£14.00]","[sterling, silver, organic, split, band, ring]"
4,"[sterling, silver, three, band, hammered, spin...",[32.00],"[sale, price£32.00]","[sterling, silver, three, band, hammered, spin..."
...,...,...,...,...
1651,"[surgical, steel, crystal, pave, disc, labret]",[6.99],"[sale, price£6.99]","[surgical, steel, and, crystal, pave, disc, la..."
1652,"[surgical, steel, synthetic, opal, mini, stone...",[4.99],"[sale, price£4.99]","[surgical, steel, synthetic, opal, mini, stone..."
1653,"[surgical, steel, opal, effect, crescent, moon...",[4.99],"[sale, price£4.99]","[surgical, steel, opal, effect, crescent, moon..."
1654,"[surgical, steel, labret, single, crystal]",[5.99],"[sale, price£5.99]","[surgical, steel, labret, with, single, crystal]"


In [85]:
# Here, I used the cleaned sale price with the untokenized product name to arrange into a new csv file.
df_cleaned = pd.DataFrame({
    'Product Name': [' '.join(tokens) for tokens in df['Product Name']],
    'Sale price': df['Sale price']})

# Save the new DataFrame to a CSV file
df_cleaned.to_csv('cleaned_data.csv', index=False, encoding='utf-8')

df = df.applymap(lambda text: ['hoop' if word=='hoops' else word for word in text])


/var/folders/1t/ywcqy6hd69g8pdsvmwf99pwh0000gn/T/ipykernel_45438/2970021412.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda text: ['hoop' if word=='hoops' else word for word in text])


# Bag of Words representation of dataset
Code adapted from: https://git.arts.ac.uk/tbroad/NLP-23-24/blob/main/Week-4-Search-and-topic-modeling/topic-modelling-bow-lda.ipynb

In [86]:
# I used a BoW representation for the  product descriptions to analyze word frequency.
documents = [' '.join(tokens) for tokens in df['tokenized_product_name']] 

vectorizer = CountVectorizer()
bag_of_words = vectorizer.fit_transform(documents)

# Update the Counter with the actual counts from the vectorizer
bow = Counter(vectorizer.get_feature_names_out())

print(f'Bag of words matrix shape and size {bag_of_words.shape}')
print(f'Number of unique words: {len(bow)}')

Bag of words matrix shape and size (1656, 144)
Number of unique words: 144


In [87]:
vocab = vectorizer.get_feature_names_out()
document_labels = df.index 

bow_df = pd.DataFrame(bag_of_words.toarray(), columns=vocab, index=document_labels)
bow_df

,10mm,12mm,1mm,2mm,35mm,3mm,4mm,5mm,7mm,8mm,...,turquoise,twist,twisted,ultra,wave,wedding,wiggle,wishbone,with,zirconia
0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1651,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1652,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1653,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1654,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


# Feature extraction using TF-IDF (Term Frequency-Inverse Document Frequency).
Code adapted from: https://git.arts.ac.uk/tbroad/NLP-23-24/blob/main/Week-4-Search-and-topic-modeling/topic-modelling-bow-lda.ipynb
find out the weight of keywords within my document to see if ring type directly influences the price of the ring.
Converting text data into numerical value.

In [72]:
print(df.columns)

Index(['Product Name', 'Sale price', 'tokenized_sale_price',
       'tokenized_product_name'],
      dtype='object')


In [73]:
# 
vocab = vectorizer.get_feature_names_out(df['tokenized_product_name'])

bow_df = pd.DataFrame(bag_of_words.toarray(), columns=vocab) # creates a DataFrame
print(bow_df)

      10mm  12mm  1mm  2mm  35mm  3mm  4mm  5mm  7mm  8mm  ...  turquoise  \
0        0     0    0    0     0    0    0    1    0    0  ...          0   
1        0     0    0    0     0    0    0    0    0    0  ...          0   
2        0     0    0    0     0    0    0    0    0    0  ...          0   
3        0     0    0    0     0    0    0    0    0    0  ...          0   
4        0     0    0    0     0    0    0    0    0    0  ...          0   
...    ...   ...  ...  ...   ...  ...  ...  ...  ...  ...  ...        ...   
1651     0     0    0    0     0    0    0    0    0    0  ...          0   
1652     0     0    0    0     0    0    0    0    0    0  ...          0   
1653     0     0    0    0     0    0    0    0    0    0  ...          0   
1654     0     0    0    0     0    0    0    0    0    0  ...          0   
1655     0     0    0    0     0    0    0    0    0    0  ...          0   

      twist  twisted  ultra  wave  wedding  wiggle  wishbone  with  zirconi

In [117]:
bow_df.iloc[90].sort_values()[-20:]

disc           0
huggie         0
daisy          0
cutout         0
curved         0
cubic          0
huggy          0
crystal        0
crossover      0
crescent       0
creole         0
heart          0
herringbone    0
shaped         1
band           1
sterling       1
plain          1
ring           1
4mm            1
silver         1
Name: 90, dtype: int64

In [75]:
# Now that I have all the product description in my bag of words matrix, I set out to find which word was the most frequent, following code from week 4 lecture to calculate the TF/IDF.
vocab = vectorizer.get_feature_names_out()
#document_labels = df.index 

#bow_df = pd.DataFrame(bag_of_words.toarray(), columns=vocab)
bag_df = pd.DataFrame(bag_of_words.todense(), columns = vocab)

# finding the total  counts for each word across all documents
word_counts = bow_df.sum()

# Finding the most frequent word
most_frequent_word = word_counts.idxmax() #code to find the most frequent in my data set, adapted from https://stackoverflow.com/questions/68891694/find-the-most-frequent-word-in-each-column-in-a-dataframe-and-combine-those-word.

count_of_most_frequent_word = word_counts.max()

print(f"The most frequent word is '{most_frequent_word}' with a count of {count_of_most_frequent_word}.")

The most frequent word is 'silver' with a count of 1575.


In [88]:
# I wanted to find out the most frequent words within my bag of words matrix to gain a better understanding of my data set, as opposed to just a single frequent word.
# First, i created an empty list from which my data would be stored
word_sums = {}

#I needed to find a way to iterate through each column in the data frame and collect the sum of each word, so I used a for loop, e.g., for each column in bag of words matrix, add sum and commit to word_sum.
for column in bow_df:
    word_sums[column] = bow_df[column].sum()

# LLM Disclaimer: I used a LLM here to work out how why i had the following error; AttributeError: 'dict' object has no attribute 'sort_values'.
# The dictionary used needed to be converted to a pandas Series for further usage.
word_sums = pd.Series(word_sums)

# Sort the series in descending order to find most common words
most_common_words = word_sums.sort_values(ascending=False)

# Once the code was debugged, the following printed out the most common words
print('The most common words are:\n', most_common_words)

The most common words are:
 sterling     1575
silver       1575
hoop          584
studs         420
ring          333
             ... 
snakey          1
pave            1
single          1
effect          1
lightning       1
Length: 144, dtype: int64


In [89]:
# As expected, 'silver' was the most frequent word, probably becuase Essential jewellry specialises in jewellery with this material type. 
# What are the top 50 most frequent words?
# Using the total counts for each word across all documents I set parameters to find the top 50 most requent words with nlargest.
word_counts = bow_df.sum()

# Finding the 50 most frequent words
top_50_words = word_counts.nlargest(50)

print("Top 50 most frequent words:")
print(top_50_words)

# Origionally, 'hoops' and 'hoop' appeared in the top_50_words but I added a stemmer to reduce word duplicates within the data cleaning process.
# This reduced the shape of my BoW matrix from 144 unique words. 'hoop' appeared higher up in the word frequency list.

Top 50 most frequent words:
silver       1575
sterling     1575
hoop          584
studs         420
ring          333
cubic         327
zirconia      327
and           296
band          182
bead          157
3mm           147
bracelet      147
10mm          126
sleeper       126
2mm           109
set           109
claw          103
tiny          103
hammered       96
opal           86
12mm           84
charm          84
gemstone       84
huggie         84
huggy          84
heart          83
with           72
mini           71
nose           70
star           62
8mm            57
steel          57
surgical       57
4mm            56
fine           56
simple         56
hinged         54
crystal        52
plain          52
open           49
tube           49
chain          47
double         47
square         47
35mm           42
celestial      42
charms         42
creole         42
crossover      42
earrings       42
dtype: int64


In [120]:
# What are the key topics within my data set?
# code adapted from https://git.arts.ac.uk/tbroad/NLP-23-24/blob/main/Week-4-Search-and-topic-modeling/topic-modelling-bow-lda.ipynb.

num_topics = 3 
lda = LatentDirichletAllocation(n_components=num_topics,random_state=123, learning_method='batch')
lda_topics = lda.fit_transform(bag_of_words)
#labels = ['topic0', 'topic1', 'topic2']

topic_weights = pd.DataFrame(lda.components_.T, index=vocab, columns=labels)
topic_weights.sample(28)

num_terms = 4 #Why this number? (consider the volume of ring data and choose an appropriate value)
for i in range(num_topics):
    print("___topic " + str(i) + "___")
    topicName = "topic" + str(i)
    weightedlist = topic_weights.get(topicName).sort_values()[-num_terms:]
    print(weightedlist.index.values)


___topic 0___
['and' 'hoop' 'silver' 'sterling']
___topic 1___
['3mm' 'studs' 'sterling' 'silver']
___topic 2___
['band' 'ring' 'sterling' 'silver']


In [121]:
# I printed out whether there were any topics within my data, then realised that my data was not suitable for LDA methods.
print(topic_weights.keys())

Index(['topic0', 'topic1', 'topic2'], dtype='object')


In [122]:
# Locating where the pricing index resides within the sorted dataframe.
price = sorted_df.iloc[:, 1]
price

249     [10.00]
57      [10.00]
1016    [10.00]
321     [10.00]
1615    [10.00]
         ...   
1544     [9.00]
1520     [9.00]
1018     [9.00]
329      [9.00]
0        [9.00]
Name: Sale price, Length: 1656, dtype: object

# Classifying product price data

Creating a function which classifes what product is cheap, affordable and expensive.
- This classification process may be subjective, contain my own opinion on price point
- Code adapted from: https://git.arts.ac.uk/tbroad/NLP-23-24/blob/main/Week-8-Word-vectors/classification-with-word-vectors.ipynb , NLP-Week-6, available at: https://git.arts.ac.uk/tbroad/NLP-23-24/blob/main/Week-6-Classification/classification-lecture.ipynb and  https://git.arts.ac.uk/tbroad/NLP-23-24/blob/main/Week-6-Classification/we-rate-dogs-sentiment-analysis.ipynb 
Critically, my data is quite small for training. What are my other options?

In [123]:
# Here, I created a classification profile for my data.
def classify_price(price):
    if price < 10: #if the jewellery is less than £10, consider this cheap.
        return 'Cheap'
    elif 10 <= price <= 30: #if the jewellery is more than or equal to £ but less than £20.
        return 'Affordable'
    else:
        return 'Expensive'

In [133]:
# Code which uses my preprocessed data and places them in an ordered list from most to least expensive ring. Will this help to identify the reltionship between the product description and price?

sorted_df = sorted_df.sort_values('Sale price', ascending=True).head(1656)
#sorted_df = sorted_df.sort_values('Sale price', ascending=True).head(1656) #add a 'true' statement to sort from lowest value to highest.
sorted_df

,Product Name,Sale price,tokenized_sale_price,tokenized_product_name
321,"[sterling, silver, narrow, hammered, band]",[10.00],"[sale, price£10.00]","[sterling, silver, narrow, hammered, band]"
153,"[sterling, silver, narrow, hammered, band]",[10.00],"[sale, price£10.00]","[sterling, silver, narrow, hammered, band]"
177,"[sterling, silver, narrow, hammered, band]",[10.00],"[sale, price£10.00]","[sterling, silver, narrow, hammered, band]"
1064,"[sterling, silver, simple, bobble, snake, chai...",[10.00],"[sale, price£10.00]","[sterling, silver, simple, bobble, snake, chai..."
201,"[sterling, silver, narrow, hammered, band]",[10.00],"[sale, price£10.00]","[sterling, silver, narrow, hammered, band]"
...,...,...,...,...
305,"[sterling, silver, cubic, zirconia, ultra, fin...",[9.00],"[sale, price£9.00]","[sterling, silver, and, cubic, zirconia, ultra..."
1138,"[sterling, silver, fine, bobble, bead, bracelet]",[9.00],"[sale, price£9.00]","[sterling, silver, fine, bobble, bead, bracelet]"
1060,"[sterling, silver, flat, star, link, bracelet]",[9.00],"[sale, price£9.00]","[sterling, silver, flat, star, link, bracelet]"
216,"[sterling, silver, tiny, 1.5mm, plain, band, r...",[9.00],"[sale, price£9.00]","[sterling, silver, tiny, 1.5mm, plain, band, r..."


In [134]:
# I used a LLM here to debug my code. I had origionally written 'sorted_df['price_category'] = sorted_df['Sale price'].apply(classify_price)', which came up with error. I asked why the code wasnt working and how i could improve, resulting on the below code.
sorted_df['Sale price'] = sorted_df['Sale price'].apply(lambda x: float(x) if x else np.nan) 

TypeError: float() argument must be a string or a number, not 'list'

In [126]:
# I printed out the results (and crossed my fingers) to see if the above code was correct
sorted_df

,Product Name,Sale price,tokenized_sale_price,tokenized_product_name
249,"[sterling, silver, narrow, hammered, band]",[10.00],"[sale, price£10.00]","[sterling, silver, narrow, hammered, band]"
1088,"[sterling, silver, simple, bobble, snake, chai...",[10.00],"[sale, price£10.00]","[sterling, silver, simple, bobble, snake, chai..."
1543,"[surgical, steel, crystal, edge, hoop]",[10.00],"[sale, pricefrom, £10.00]","[surgical, steel, crystal, edge, hoop]"
225,"[sterling, silver, narrow, hammered, band]",[10.00],"[sale, price£10.00]","[sterling, silver, narrow, hammered, band]"
9,"[sterling, silver, narrow, hammered, band]",[10.00],"[sale, price£10.00]","[sterling, silver, narrow, hammered, band]"
...,...,...,...,...
44,"[sterling, silver, little, wiggle, ring]",[9.00],"[sale, price£9.00]","[sterling, silver, little, wiggle, ring]"
240,"[sterling, silver, tiny, 1.5mm, plain, band, r...",[9.00],"[sale, price£9.00]","[sterling, silver, tiny, 1.5mm, plain, band, r..."
332,"[sterling, silver, little, wiggle, ring]",[9.00],"[sale, price£9.00]","[sterling, silver, little, wiggle, ring]"
216,"[sterling, silver, tiny, 1.5mm, plain, band, r...",[9.00],"[sale, price£9.00]","[sterling, silver, tiny, 1.5mm, plain, band, r..."


In [127]:
# What are the most expensive jewellery types within my dataframe? 
#Convert order of code  to descending 
sorted_df = df.sort_values('Sale price', ascending=False).head(1656)
most_expensive_jewellery =sorted_df.iloc[0]
most_expensive_jewellery


Product Name              [sterling, silver, tiny, 1.5mm, plain, band, r...
Sale price                                                           [9.00]
tokenized_sale_price                                     [sale, price£9.00]
tokenized_product_name    [sterling, silver, tiny, 1.5mm, plain, band, r...
Name: 0, dtype: object

In [128]:
top_50_expensive_jewellery= sorted_df.head(10)
top_50_expensive_jewellery

,Product Name,Sale price,tokenized_sale_price,tokenized_product_name
0,"[sterling, silver, tiny, 1.5mm, plain, band, r...",[9.00],"[sale, price£9.00]","[sterling, silver, tiny, 1.5mm, plain, band, r..."
65,"[sterling, silver, cubic, zirconia, ultra, fin...",[9.00],"[sale, price£9.00]","[sterling, silver, and, cubic, zirconia, ultra..."
1138,"[sterling, silver, fine, bobble, bead, bracelet]",[9.00],"[sale, price£9.00]","[sterling, silver, fine, bobble, bead, bracelet]"
1132,"[sterling, silver, flat, star, link, bracelet]",[9.00],"[sale, price£9.00]","[sterling, silver, flat, star, link, bracelet]"
1122,"[sterling, silver, fine, chain, ball, bracelet...",[9.00],"[sale, price£9.00]","[sterling, silver, fine, chain, and, ball, bra..."
1114,"[sterling, silver, fine, bobble, bead, bracelet]",[9.00],"[sale, price£9.00]","[sterling, silver, fine, bobble, bead, bracelet]"
1108,"[sterling, silver, flat, star, link, bracelet]",[9.00],"[sale, price£9.00]","[sterling, silver, flat, star, link, bracelet]"
1098,"[sterling, silver, fine, chain, ball, bracelet...",[9.00],"[sale, price£9.00]","[sterling, silver, fine, chain, and, ball, bra..."
1090,"[sterling, silver, fine, bobble, bead, bracelet]",[9.00],"[sale, price£9.00]","[sterling, silver, fine, bobble, bead, bracelet]"
1084,"[sterling, silver, flat, star, link, bracelet]",[9.00],"[sale, price£9.00]","[sterling, silver, flat, star, link, bracelet]"


# Citation list : #

- Classification/classification-lecture. Available at: https://git.arts.ac.uk/tbroad/NLP-23-24/blob/main/Week-6-Classification/classification-lecture.ipynb. [Accessed 16 Nov.2023]
- Eslam Elgourany (2019). with file.open(‘r’,encoding=‘utf-8’) as f: AttributeError: ‘str’ object has no attribute ‘open’.Stack Overflow. Available at: https://stackoverflow.com/questions/59422448/with-file-openr-encoding-utf-8-as-f-attributeerror-str-object-has-no-a [Accessed 13 Nov. 2023].
- Essential Jewellery.(2023). Sterling Silver and Cubic Zirconia Marquis Wishbone Ring.Available at: https://essentialjewellery.com/products/sterling-silver-and-cubic-zirconia-marquis-wishbone-ring?_pos=4&_sid=7c1e45be0&_ss=r [Accessed 1 Nov. 2023].
- John Watson Rooney (2020). User Agent Switching - Python Web Scraping. YouTube. Available at: https://www.youtube.com/watch?v=90t9WkQbQ2E [Accessed 28 Nov. 2023].
- lyo canty. (2021). Find the most frequent word in each column in a Dataframe and combine those words into a sentence. Stack Overflow. Available at: https://stackoverflow.com/questions/68891694/find-the-most-frequent-word-in-each-column-in-a-dataframe-and-combine-those-word [Accessed 27 Nov. 2023].
- Manipulating text/bonus-excercise. Available at: https://git.arts.ac.uk/tbroad/NLP-23-24/blob/main/Week-3-Manipulating-text/bonus-exercise-regex.ipynb. [Accessed 4 Nov. 2023]
- Manipulating text/stemmer and stop-words. Available at: https://git.arts.ac.uk/tbroad/NLP-23-24/blob/main/Week-3-Manipulating-text/stemmer-and-stop-words.ipynb.
- Mohan, M. (2020). Web Scraping Python Tutorial – How to Scrape Data From A Website. freeCodeCamp.org. Available at: https://www.freecodecamp.org/news/web-scraping-python-tutorial-how-to-scrape-data-from-a-website/ [Accessed 9 Nov. 2023].
- Pandora(2023). Classic Wishbone Ring. Available at: https://uk.pandora.net/en/rings/rings/stackable-rings/classic-wishbone-ring/197790CZ.html#navigation [Accessed 8 Nov. 2023].
- Paul, T. (2022). Web Scraping in Python – How to Scrape an eCommerce Website with 20 lines of Python code. Datahut Blog. Available at: https://www.blog.datahut.co/post/web-scraping-in-python [Accessed 15 Nov. 2023].
- Ramadan, M. (2023). User Agents for Web Scraping. ScrapFly Blog. Available at: https://scrapfly.io/blog/user-agent-header-in-web-scraping/ [Accessed 12 Nov. 2023].
why-pandas-dataframe-sumaxis-0-returns-sum-of-values-in-each-column-where-axis [Accessed 5 Dec. 2023].
- STEM Week 6 Images.Available at: https://git.arts.ac.uk/lmccallum/STEM-4-Creatives-23-24/blob/main/STEM-Week-6-Images.ipynb. [Accessed 20 Nov. 2023]
- Text as numbers. Available at: https://git.arts.ac.uk/tbroad/NLP-23-24/blob/main//NLP%20Week%202.1-Text%20as%20Numbers-class.ipynb. [Accessed 14 Nov. 2023].
- Topic-modelling-bow-lda. Available at: https://git.arts.ac.uk/tbroad/NLP-23-24/blob/main/Week-4-Search-and-topic-modeling/topic-modelling-bow-lda.ipynb. [Accessed 25 Nov. 2023].
- Web scrape nursery rhymes.Available at: https://git.arts.ac.uk/tbroad/web-scrape-nursery-rhymes/commit/da77acf4524ab39b2476c1a6d5feddad888375f9. [Accessed 2 Nov. 2023]

